In [1]:
import random
import math

In [7]:
def readInput(filename):
    input = open(filename, "r")    
    nbResources, nbUsers = [int(i) for i in input.readline().split()]
    storages = []
    fixedCost = []
    demands = []
    cost = []
    for i in range(nbResources):
        x, y = [int(i) for i in input.readline().split()]
        storages.append(x)
        fixedCost.append(y)
    for i in range(nbUsers):
        x, _ = [int(i) for i in input.readline().split()]
        demands.append(x)
        cost.append([float(j) for j in input.readline().split()])
    
    return nbUsers, nbResources, cost, fixedCost, demands, storages

'''def readInput(filename):
    input = open(filename, "r")    
    nbUsers, nbResources = [int(i) for i in input.readline().split()]    
    demands = [int(i) for i in input.readline().split()]    
    cost = [[int(j) for j in input.readline().split()] for i in range(nbUsers)]    
    fixedCost = [int(j) for j in input.readline().split()]    
    storages = [int(i) for i in input.readline().split()]
    
    return nbUsers, nbResources, cost, fixedCost, demands, storages
'''

def isFeasible(nbResources, solution):
    for j in range(nbResources):
        if solution[j]:
            return True
    return False


def initialize(nbResources, probability):
    solution = []
    for j in range(nbResources):
        solution.append(random.random() < probability)
        
    if not isFeasible(nbResources, solution):
        solution[random.randrange(nbResources)] = True
        
    return solution

def invert(nbResources, solution):
    j = random.randrange(nbResources)
    solution[j] = not solution[j]
    if isFeasible(nbResources, solution):
        return j
    solution[j] = not solution[j]
    return -1

def restore(j, solution):
    solution[j] = not solution[j]

    
def restoreStorages(changes):
    for i in changes:
        storages[changes[i]] += demands[i]

def solutionValue(nbUsers, nbResources, cost, fixedCost,\
                  solution, demands, storages):
    globalValue = float('inf')
    value = 0.0
    used = [False] * nbResources
    changes = {}
    
    for i in range(math.ceil(math.sqrt(nbUsers))):
        users = shuffleUsers(nbUsers)
    
        for i in users:
            minCost = float('inf')
            jUsed = -1
            for j in range(nbResources):
                if (solution[j] and cost[i][j] < minCost) and \
                        (demands[i] <= storages[j]):
                    minCost = cost[i][j]
                    jUsed = j

            if (jUsed == -1):
                restoreStorages(changes)
                return False

            value += minCost
            storages[jUsed] -= demands[i]
            used[jUsed] = True

            changes[i] = jUsed

        for j in range(nbResources):
            if used[j]:
                value += fixedCost[j]
                
        
        solution = used
        restoreStorages(changes)
        
        if value < globalValue:
            globalValue = value
    
    return globalValue


def tabuSearch(nbUsers, nbResources, cost, fixedCost,\
               demands, storages, solution, maxIters, tabuListLength):
    
    tabooOutput = 0
    for i in range(maxIters):
        currValue = solutionValue(nbUsers, nbResources, cost, fixedCost, solution, demands, storages)
        if(not currValue):
            continue
        break
    
    if (currValue == False): # i == maxIters
        print('Starting point not found!\n')
        return False
    
    bestValue = currValue
   
    i = 0
    T = []
    T.append(solution)

    while (i < maxIters):
        j = invert(nbResources, solution)
        if j < 0:
            continue
            
        if solution in T:
            i += 1
            continue
        
        newValue = solutionValue(nbUsers, nbResources, cost, fixedCost, solution, demands, storages)

        
        if (newValue < currValue and newValue):
            currValue = newValue
        else:
            restore(j, solution)
            
        if (newValue < bestValue and newValue):
            bestValue = newValue
            if(len(T) > tabuListLength):
                T.remove(T[0])
                tabooOutput += 1
            T.append(solution)
            
        i += 1
        
    return bestValue, tabooOutput

In [8]:
def shuffleUsers(nbUsers):
    
    users = [int(i) for i in range(nbUsers)]
    
    random.shuffle(users)
    
    return users

In [10]:
#nbUsers, nbResources, cost, fixedCost, demands, storages = readInput('input_cap_test')
nbUsers, nbResources, cost, fixedCost, demands, storages = readInput('cap131')
#nbUsers = 10
#nbResources = 10
#print(nbUsers)
#print(nbResources)
#print(shuffleUsers(nbUsers))
#print('Fixed cost\\n', fixedCost)
#print('Demands\\n', demands)
#print('Cost\\n{}'.format(cost))
#print('Storages\\n', storages)
#print(demands)
#print(sum(demands))
prime = float('inf')

for i in range(500):
    solution = initialize(nbResources, 1)
    bestValue, output = tabuSearch(nbUsers, nbResources, cost , fixedCost, demands, storages, solution, 40000, 5)
    if bestValue < prime:
        prime = bestValue
        
print(output)
print('Best value:', prime)

0
Best value: 982195.3249999997
